In [95]:
import pandas as pd
import numpy as np
import yfinance as yf
import itertools

# Useful tool

In [418]:
def get_stock(ticker):
    # Get data on this ticker
    tickerData = yf.Ticker(ticker)

    # Get the historical prices for this ticker
    stocks = tickerData.history(period='1d', start='2021-1-1', end='2023-1-1')

    stocks.reset_index(inplace=True)
    stocks.Date = pd.to_datetime(stocks.Date).dt.date
    return stocks

In [419]:
stock = 'GS'
path = "/Users/tony/Desktop/Antoine/Semestre/9.Financial ML/ProjectFinancialNews/data/GS_roberta_output.csv"
stocks = get_stock(stock)

In [420]:
def cumulative_reward_long(df, func):
    a = func(df)
    buy_signal = a[0]
    sell_signal = a[1]
    buy_signal = [x for x in buy_signal if x is not np.nan]
    sell_signal = [x for x in sell_signal if x is not np.nan]

    # Assume we buy and sell at the close of the day
    stock_Return = []
    cash = 100  # Initial cash amount
    for i in range(0, len(sell_signal)):
        shares_to_buy = cash / buy_signal[i]
        new_cash = shares_to_buy * sell_signal[i]
        stock_Return.append(new_cash - cash)
        cash = new_cash
    
    return np.sum(stock_Return), np.std(stock_Return)

In [421]:
def cumulative_reward_short(df, func):
    a = func(df)
    short_signal = a[0]
    cover_signal = a[1]
    short_signal = [x for x in short_signal if x is not np.nan]
    cover_signal = [x for x in cover_signal if x is not np.nan]

    # Assume we sell short and then cover at the close of the day
    stock_Return = []
    cash = 100  # Initial cash amount
    for i in range(0, len(cover_signal)):
        shares_to_short = cash / short_signal[i]
        new_cash = shares_to_short * cover_signal[i]
        stock_Return.append(new_cash - cash)
        cash = new_cash
    
    return np.sum(stock_Return), np.std(stock_Return)

In [422]:
def split_into_array(s):
    # Remove the leading and trailing square brackets and single quotes
    s = s[1:-1]

    # Split the string into a list of sentences
    integers = s.split(", ")

    return integers

def merge_arrays(series):
    return list(itertools.chain(*series.tolist()))

def prepare_dataset(path, columns):
    df = pd.read_csv(path)
    
    df['Predictions'] = df['Predictions'].apply(split_into_array).apply(np.array).apply(lambda x: x.astype(int))
    df['Positive'] = df['Positive'].apply(split_into_array).apply(np.array).apply(lambda x: x.astype(float))
    df['Negative'] = df['Negative'].apply(split_into_array).apply(np.array).apply(lambda x: x.astype(float))
    df['Neutral'] = df['Neutral'].apply(split_into_array).apply(np.array).apply(lambda x: x.astype(float))
    df['published'] = pd.to_datetime(df['published']).dt.date
    
    columns_df = columns.copy()
    columns_df.insert(0, 'published')
    
    input = df[columns_df]
    input = input.copy()
    input.rename(columns={'published': 'Date'}, inplace=True)
    
    grouped_df = input.groupby('Date')[columns].agg(merge_arrays)    
    grouped_df = pd.merge(grouped_df, stocks[['Date', 'Close']], on='Date', how='left')

    grouped_df = grouped_df.dropna()
    grouped_df = grouped_df.reset_index(drop=True)

    return grouped_df

# Simple positive negative strategys

In [423]:
def buy_sell_sentiment(signal):
    Buy = [np.nan for i in range(0, len(signal))]
    Sell = [np.nan for i in range(0, len(signal))]
    flag = 0

    for i in range(0, len(signal)):
        if signal['Predictions'][i] == 0:  # Positive sentiment
            if flag != 1:
                Buy[i] = signal['Close'][i]
                flag = 1
        elif signal['Predictions'][i] == 1:  # Negative sentiment
            if flag != 0:
                Sell[i] = signal['Close'][i]
                flag = 0
    return (Buy, Sell)

In [424]:
def short_sell_sentiment(signal):
    Short = [np.nan for i in range(0, len(signal))]
    Sell = [np.nan for i in range(0, len(signal))]
    flag = 0

    for i in range(0, len(signal)):
        if signal['Predictions'][i] == 1:  # Negative sentiment
            if flag != 1:
                Short[i] = signal['Close'][i]
                flag = 1
        elif signal['Predictions'][i] == 0:  # Positive sentiment
            if flag != 0:
                Sell[i] = signal['Close'][i]
                flag = 0
    return (Short, Sell)

Simple first implementation we look at the sentiment of the news and if it is positive we buy and if it is negative we sell. We will use the sentiment of the news from the previous day to make our decision, by mean of the sentences in a same day.

In [425]:
simple_news_dataset = prepare_dataset(path, ['Predictions'])

In [426]:
simple_news_dataset

,Date,Predictions,Close
0,2021-01-04,"[0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0]",245.939194
1,2021-01-05,"[0, 0, 0, 0, 2, 0, 1]",251.442734
2,2021-01-06,"[1, 2, 2, 2, 1, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0]",265.011108
3,2021-01-07,"[2, 0, 1, 2, 2, 0, 0, 0, 0]",270.672363
4,2021-01-08,"[0, 2, 2, 1, 0, 2, 2, 0, 0, 2, 2]",269.215302
...,...,...,...
497,2022-12-23,"[1, 2, 1, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 1, 2, ...",334.895905
498,2022-12-27,"[1, 0, 1, 1, 0, 2, 1, 1, 2, 0, 1, 0, 1, 0, 1, ...",331.464630
499,2022-12-28,"[2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 1, 1, 0, 1, 2, ...",330.398438
500,2022-12-29,"[0, 2, 2, 1, 1, 2, 0, 1, 2, 2, 2, 2, 2, 2, 1, ...",332.879791


In [427]:
from collections import Counter

def take_max_count_prediction(lst):
    counts = Counter(lst)
    max_count = max(counts.values())
    modes = [k for k, v in counts.items() if v == max_count]
    
    if len(modes) == 1:
        return modes[0]
    elif len(modes) == 3:
        return 2
    elif 0 in modes and 1 in modes:
        return 2
    elif 0 in modes and 2 in modes:
        return 0
    elif 1 in modes and 2 in modes:
        return 1
    else:
        return 2

simple_news_dataset['Predictions'] = simple_news_dataset['Predictions'].apply(take_max_count_prediction)

In [428]:
simple_news_dataset.head()

,Date,Predictions,Close
0,2021-01-04,2,245.939194
1,2021-01-05,0,251.442734
2,2021-01-06,2,265.011108
3,2021-01-07,0,270.672363
4,2021-01-08,2,269.215302


In [429]:
print("Long Cumulative Return : ", cumulative_reward_long(simple_news_dataset, buy_sell_sentiment))
print("Short Cumulative Return : ", cumulative_reward_short(simple_news_dataset, short_sell_sentiment))

Long Cumulative Return :  (21.717359539388895, 4.9899999114490905)
Short Cumulative Return :  (8.750831770198062, 5.381610149173918)


Now we take the maximum of the different sentences per day between positive and Negative

In [430]:
max_sum_score = prepare_dataset(path, ['Positive', 'Negative', 'Neutral'])
max_sum_score.head()

,Date,Positive,Negative,Neutral,Close
0,2021-01-04,"[0.9996306896209717, 4.568207441479899e-05, 5....","[0.00019008245726581663, 7.416177686536685e-05...","[0.00017915238277055323, 0.9998801946640015, 0...",245.939194
1,2021-01-05,"[0.6521651148796082, 0.9712276458740234, 0.999...","[0.0004671250644605607, 0.004945231601595879, ...","[0.3473678231239319, 0.023827143013477325, 0.0...",251.442734
2,2021-01-06,"[0.0011304820654913783, 5.7131750509142876e-05...","[0.9983174800872803, 7.38463131710887e-05, 7.1...","[0.0005520224221982062, 0.9998689889907837, 0....",265.011108
3,2021-01-07,"[5.761160355177708e-05, 0.9996683597564697, 0....","[8.459905802737921e-05, 0.00019280854030512273...","[0.9998577833175659, 0.00013889373803976923, 0...",270.672363
4,2021-01-08,"[0.9997280240058899, 5.761160355177708e-05, 5....","[0.0001962340174941346, 8.459905802737921e-05,...","[7.569414447061718e-05, 0.9998577833175659, 0....",269.215302


In [431]:
max_sum_score[['Positive', 'Negative', 'Neutral']] = max_sum_score[['Positive', 'Negative', 'Neutral']].applymap(sum)
max_sum_score.head()

,Date,Positive,Negative,Neutral,Close
0,2021-01-04,3.724697,0.001931,11.273373,245.939194
1,2021-01-05,4.885495,1.005012,1.109493,251.442734
2,2021-01-06,2.923271,2.003469,11.073260,265.011108
3,2021-01-07,4.532861,1.007042,3.460097,270.672363
4,2021-01-08,3.996236,1.000273,6.003491,269.215302


In [432]:
# Python
def take_max_count_prediction(row):
    values = {'Positive': row['Positive'], 'Negative': row['Negative'], 'Neutral': row['Neutral']}
    max_key = max(values, key=values.get)
    
    if max_key == 'Positive':
        return 0
    elif max_key == 'Negative':
        return 1
    else:
        return 2

max_sum_score['Predictions'] = max_sum_score.apply(take_max_count_prediction, axis=1)

In [433]:
max_sum_score.head()

,Date,Positive,Negative,Neutral,Close,Predictions
0,2021-01-04,3.724697,0.001931,11.273373,245.939194,2
1,2021-01-05,4.885495,1.005012,1.109493,251.442734,0
2,2021-01-06,2.923271,2.003469,11.073260,265.011108,2
3,2021-01-07,4.532861,1.007042,3.460097,270.672363,0
4,2021-01-08,3.996236,1.000273,6.003491,269.215302,2


In [434]:
print("Long Cumulative Return : ", cumulative_reward_long(max_sum_score, buy_sell_sentiment))
print("Short Cumulative Return : ", cumulative_reward_short(max_sum_score, short_sell_sentiment))

Long Cumulative Return :  (16.496980317182434, 5.572957256072591)
Short Cumulative Return :  (13.624096133146395, 5.718298921053837)


We perform less with this technique

We take some threshold only the value greater than this into account, with mean of the day

In [435]:
threshold_input = prepare_dataset(path, ['Positive', 'Negative', 'Neutral'])
THRESHOLD = 0.5

In [436]:
def replace_below_threshold(arr, threshold):
    return [x for x in arr if x >= threshold]

columns = ['Positive', 'Negative', 'Neutral']

for col in columns:
    threshold_input[col] = threshold_input[col].apply(lambda arr: replace_below_threshold(arr, threshold=THRESHOLD))

In [437]:
threshold_input.head()

,Date,Positive,Negative,Neutral,Close
0,2021-01-04,"[0.9996306896209717, 0.8621811866760254, 0.862...",[],"[0.9998801946640015, 0.9998689889907837, 0.999...",245.939194
1,2021-01-05,"[0.6521651148796082, 0.9712276458740234, 0.999...",[0.9980467557907104],[0.7367658615112305],251.442734
2,2021-01-06,"[0.9186276197433472, 0.9996577501296997, 0.988...","[0.9983174800872803, 0.9986239671707153]","[0.9998689889907837, 0.9998816251754761, 0.999...",265.011108
3,2021-01-07,"[0.9996683597564697, 0.9996199607849121, 0.991...",[0.9970082640647888],"[0.9998577833175659, 0.9998613595962524, 0.999...",270.672363
4,2021-01-08,"[0.9997280240058899, 0.9962294697761536, 0.999...",[0.9985669255256653],"[0.9998577833175659, 0.9998577833175659, 0.999...",269.215302


In [438]:
threshold_input[['Positive', 'Negative', 'Neutral']] = threshold_input[['Positive', 'Negative', 'Neutral']].applymap(len)

In [439]:
threshold_input.head()

,Date,Positive,Negative,Neutral,Close
0,2021-01-04,4,0,11,245.939194
1,2021-01-05,5,1,1,251.442734
2,2021-01-06,3,2,11,265.011108
3,2021-01-07,5,1,3,270.672363
4,2021-01-08,4,1,6,269.215302


In [440]:
threshold_input['Predictions'] = threshold_input.apply(take_max_count_prediction, axis=1)

In [441]:
threshold_input.head()

,Date,Positive,Negative,Neutral,Close,Predictions
0,2021-01-04,4,0,11,245.939194,2
1,2021-01-05,5,1,1,251.442734,0
2,2021-01-06,3,2,11,265.011108,2
3,2021-01-07,5,1,3,270.672363,0
4,2021-01-08,4,1,6,269.215302,2


In [442]:
print("Long Cumulative Return : ", cumulative_reward_long(threshold_input, buy_sell_sentiment))
print("Short Cumulative Return : ", cumulative_reward_short(threshold_input, short_sell_sentiment))

Long Cumulative Return :  (19.8080411417166, 4.9542739524073856)
Short Cumulative Return :  (10.483937176832626, 5.411086168195351)


In [443]:
def threshold_test():
    THRESHOLD_ARRAY = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.85, 0.9, 0.95, 0.99]
    for THRESHOLD in THRESHOLD_ARRAY:
        threshold_input = prepare_dataset(path, ['Positive', 'Negative', 'Neutral'])
        
        columns = ['Positive', 'Negative', 'Neutral']
        for col in columns:
            threshold_input[col] = threshold_input[col].apply(lambda arr: replace_below_threshold(arr, threshold=THRESHOLD))        
            
        threshold_input[['Positive', 'Negative', 'Neutral']] = threshold_input[['Positive', 'Negative', 'Neutral']].applymap(len)
        
        threshold_input['Predictions'] = threshold_input.apply(take_max_count_prediction, axis=1)
        
        print("Threshold : ", THRESHOLD)
        print("Long Cumulative Return : ", cumulative_reward_long(threshold_input, buy_sell_sentiment))
        print("Short Cumulative Return : ", cumulative_reward_short(threshold_input, short_sell_sentiment))

In [444]:
threshold_test()

Threshold :  0.1
Long Cumulative Return :  (26.58019686356552, 5.6070690180408755)
Short Cumulative Return :  (4.572945996032516, 4.84948246556283)
Threshold :  0.2
Long Cumulative Return :  (16.564101765963883, 5.283489533657222)
Short Cumulative Return :  (13.558667636436013, 5.254050128053691)
Threshold :  0.3
Long Cumulative Return :  (21.128968444119153, 5.203636000463345)
Short Cumulative Return :  (9.279095337853931, 5.1419306267340605)
Threshold :  0.4
Long Cumulative Return :  (19.8080411417166, 4.9542739524073856)
Short Cumulative Return :  (10.483937176832626, 5.411086168195351)
Threshold :  0.5
Long Cumulative Return :  (19.8080411417166, 4.9542739524073856)
Short Cumulative Return :  (10.483937176832626, 5.411086168195351)
Threshold :  0.6
Long Cumulative Return :  (21.04526685187257, 5.363990269976076)
Short Cumulative Return :  (9.35466074009507, 5.465265283633173)
Threshold :  0.7
Long Cumulative Return :  (27.00314676640376, 5.744475914636586)
Short Cumulative Return :

Weird, better is the filter the worst is the result

# Momemntum X News trading strategy

We are going to use the better we saw at this stage the Threshold 0.7

## With MACD

## AND need both signals to be positive

In [445]:
def buy_sell_momentum_sentiment(signal):
    Buy = [np.nan for i in range(0, len(signal))]
    Sell = [np.nan for i in range(0, len(signal))]
    flag = 0

    for i in range(0, len(signal)):
        if signal['MACD'][i] > signal['Signal Line'][i] and signal['Predictions'][i] == 0:  # Positive momentum and sentiment
            if flag != 1:
                Buy[i] = signal['Close'][i]
                flag = 1
        elif signal['MACD'][i] < signal['Signal Line'][i] and signal['Predictions'][i] == 1:  # Negative momentum and sentiment
            if flag != 0:
                Sell[i] = signal['Close'][i]
                flag = 0

    return (Buy, Sell)

In [446]:
def short_sell_momentum_sentiment(signal):
    Short = [np.nan for i in range(0, len(signal))]
    Sell = [np.nan for i in range(0, len(signal))]
    flag = 0

    for i in range(0, len(signal)):
        if signal['MACD'][i] < signal['Signal Line'][i] and signal['Predictions'][i] == 1:  # Negative momentum and sentiment
            if flag != 1:
                Short[i] = signal['Close'][i]
                flag = 1
        elif signal['MACD'][i] > signal['Signal Line'][i] and signal['Predictions'][i] == 0:  # Positive momentum and sentiment
            if flag != 0:
                Sell[i] = signal['Close'][i]
                flag = 0

    return (Short, Sell)

In [447]:
stocks = get_stock(stock)

In [448]:
# Calculate Short Term Exponential Moving Average
ShortEMA = stocks.Close.ewm(span=12, adjust=False).mean() 
# Calculate Long Term Exponential Moving Average
LongEMA = stocks.Close.ewm(span=26, adjust=False).mean() 
# Calculate MACD line
MACD = ShortEMA - LongEMA
# Calculate Signal line
signal = MACD.ewm(span=9, adjust=False).mean()

In [449]:
stocks['MACD'] = MACD
stocks['Signal Line'] = signal

In [450]:
THRESHOLD = 0.7

MACD_news_input = prepare_dataset(path, ['Positive', 'Negative', 'Neutral'])
MACD_news_input = pd.merge(MACD_news_input, stocks[['Date', 'MACD', 'Signal Line']], on='Date', how='left')

columns = ['Positive', 'Negative', 'Neutral']

for col in columns:
    MACD_news_input[col] = MACD_news_input[col].apply(lambda arr: replace_below_threshold(arr, threshold=THRESHOLD))        
    
MACD_news_input[['Positive', 'Negative', 'Neutral']] = MACD_news_input[['Positive', 'Negative', 'Neutral']].applymap(len)

MACD_news_input['Predictions'] = MACD_news_input.apply(take_max_count_prediction, axis=1)

print("Threshold : ", THRESHOLD, " Long Cumulative Return : ", cumulative_reward_long(MACD_news_input, buy_sell_momentum_sentiment))
print("Threshold : ", THRESHOLD, " Short Cumulative Return : ", cumulative_reward_short(MACD_news_input, short_sell_momentum_sentiment))


Threshold :  0.7  Long Cumulative Return :  (18.85853601208281, 11.84563027117163)
Threshold :  0.7  Short Cumulative Return :  (23.200779575907674, 5.77044516901593)


## OR need one of the signal to be positive

In [451]:
def buy_sell_momentum_sentiment(signal):
    Buy = [np.nan for i in range(0, len(signal))]
    Sell = [np.nan for i in range(0, len(signal))]
    flag = 0

    for i in range(0, len(signal)):
        if signal['MACD'][i] > signal['Signal Line'][i] or signal['Predictions'][i] == 0:  # Positive momentum and sentiment
            if flag != 1:
                Buy[i] = signal['Close'][i]
                flag = 1
        elif signal['MACD'][i] < signal['Signal Line'][i] or signal['Predictions'][i] == 1:  # Negative momentum and sentiment
            if flag != 0:
                Sell[i] = signal['Close'][i]
                flag = 0

    return (Buy, Sell)

In [452]:
def short_sell_momentum_sentiment(signal):
    Short = [np.nan for i in range(0, len(signal))]
    Sell = [np.nan for i in range(0, len(signal))]
    flag = 0

    for i in range(0, len(signal)):
        if signal['MACD'][i] < signal['Signal Line'][i] or signal['Predictions'][i] == 1:  # Negative momentum and sentiment
            if flag != 1:
                Short[i] = signal['Close'][i]
                flag = 1
        elif signal['MACD'][i] > signal['Signal Line'][i] or signal['Predictions'][i] == 0:  # Positive momentum and sentiment
            if flag != 0:
                Sell[i] = signal['Close'][i]
                flag = 0

    return (Short, Sell)

In [453]:
stocks = get_stock(stock)

In [454]:
# Calculate Short Term Exponential Moving Average
ShortEMA = stocks.Close.ewm(span=12, adjust=False).mean() 
# Calculate Long Term Exponential Moving Average
LongEMA = stocks.Close.ewm(span=26, adjust=False).mean() 
# Calculate MACD line
MACD = ShortEMA - LongEMA
# Calculate Signal line
signal = MACD.ewm(span=9, adjust=False).mean()

In [455]:
stocks['MACD'] = MACD
stocks['Signal Line'] = signal

In [456]:
THRESHOLD = 0.7

MACD_news_input = prepare_dataset(path, ['Positive', 'Negative', 'Neutral'])
MACD_news_input = pd.merge(MACD_news_input, stocks[['Date', 'MACD', 'Signal Line']], on='Date', how='left')

columns = ['Positive', 'Negative', 'Neutral']

for col in columns:
    MACD_news_input[col] = MACD_news_input[col].apply(lambda arr: replace_below_threshold(arr, threshold=THRESHOLD))        
    
MACD_news_input[['Positive', 'Negative', 'Neutral']] = MACD_news_input[['Positive', 'Negative', 'Neutral']].applymap(len)

MACD_news_input['Predictions'] = MACD_news_input.apply(take_max_count_prediction, axis=1)

print("Threshold : ", THRESHOLD, " Long Cumulative Return : ", cumulative_reward_long(MACD_news_input, buy_sell_momentum_sentiment))
print("Threshold : ", THRESHOLD, " Short Cumulative Return : ", cumulative_reward_short(MACD_news_input, short_sell_momentum_sentiment))

Threshold :  0.7  Long Cumulative Return :  (22.649220551363342, 6.41428365612634)
Threshold :  0.7  Short Cumulative Return :  (15.177238532207184, 3.8741521341296568)


## With RSI

## AND need both signals to be positive

In [457]:
def buy_sell_RSI_sentiment(signal, upper=70, lower=30):
    Buy = [np.nan for i in range(0, len(signal))]
    Sell = [np.nan for i in range(0, len(signal))]
    flag = 0

    for i in range(0, len(signal)):
        if signal['RSI'][i] < lower and signal['Predictions'][i] == 0:
            if flag != 1:
                Buy[i] = signal['Close'][i]
                flag = 1
        elif signal['RSI'][i] > upper and signal['Predictions'][i] == 1:
            if flag != 0:
                Sell[i] = signal['Close'][i]
                flag = 0

    return (Buy, Sell)

In [458]:
def short_sell_RSI_sentiment(signal, upper=70, lower=30):
    Short = [np.nan for i in range(0, len(signal))]
    Sell = [np.nan for i in range(0, len(signal))]
    flag = 0

    for i in range(0, len(signal)):
        if signal['RSI'][i] > upper and signal['Predictions'][i] == 1:
            if flag != 1:
                Short[i] = signal['Close'][i]
                flag = 1
        elif signal['RSI'][i] < lower and signal['Predictions'][i] == 0:
            if flag != 0:
                Sell[i] = signal['Close'][i]
                flag = 0

    return (Short, Sell)

In [459]:
def calculate_RSI(data, time_window):
    diff = data.diff(1).dropna()        # diff in one field(one day)

    #this preservers dimensions off diff values
    up_chg = 0 * diff
    down_chg = 0 * diff
    
    # up change is equal to the positive difference, otherwise equal to zero
    up_chg[diff > 0] = diff[ diff>0 ]
    
    # down change is equal to negative deifference, otherwise equal to zero
    down_chg[diff < 0] = diff[ diff < 0 ]

    # we set com=time_window-1 so we get decay alpha=1/time_window
    up_chg_avg   = up_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    return rsi

stocks = get_stock(stock)

stocks['RSI'] = calculate_RSI(stocks['Close'], 14)

In [460]:
THRESHOLD = 0.7

RSI_news_input = prepare_dataset(path, ['Positive', 'Negative', 'Neutral'])
RSI_news_input = pd.merge(RSI_news_input, stocks[['Date', 'RSI']], on='Date', how='left')

columns = ['Positive', 'Negative', 'Neutral']

for col in columns:
    RSI_news_input[col] = RSI_news_input[col].apply(lambda arr: replace_below_threshold(arr, threshold=THRESHOLD))        
    
RSI_news_input[['Positive', 'Negative', 'Neutral']] = RSI_news_input[['Positive', 'Negative', 'Neutral']].applymap(len)

RSI_news_input['Predictions'] = RSI_news_input.apply(take_max_count_prediction, axis=1)

print("Threshold : ", THRESHOLD, " Long Cumulative Return : ", cumulative_reward_long(RSI_news_input, buy_sell_RSI_sentiment))
print("Threshold : ", THRESHOLD, " Short Cumulative Return : ", cumulative_reward_short(RSI_news_input, short_sell_RSI_sentiment))

Threshold :  0.7  Long Cumulative Return :  (1.3774077257499755, 0.0)
Threshold :  0.7  Short Cumulative Return :  (-3.807867729931303, 0.0)


## OR need one of the signal to be positive

In [461]:
def buy_sell_RSI_sentiment(signal, upper=70, lower=30):
    Buy = [np.nan for i in range(0, len(signal))]
    Sell = [np.nan for i in range(0, len(signal))]
    flag = 0

    for i in range(0, len(signal)):
        if signal['RSI'][i] < lower or signal['Predictions'][i] == 0:
            if flag != 1:
                Buy[i] = signal['Close'][i]
                flag = 1
        elif signal['RSI'][i] > upper or signal['Predictions'][i] == 1:
            if flag != 0:
                Sell[i] = signal['Close'][i]
                flag = 0

    return (Buy, Sell)

In [462]:
def short_sell_RSI_sentiment(signal, upper=70, lower=30):
    Short = [np.nan for i in range(0, len(signal))]
    Sell = [np.nan for i in range(0, len(signal))]
    flag = 0

    for i in range(0, len(signal)):
        if signal['RSI'][i] > upper or signal['Predictions'][i] == 1:
            if flag != 1:
                Short[i] = signal['Close'][i]
                flag = 1
        elif signal['RSI'][i] < lower or signal['Predictions'][i] == 0:
            if flag != 0:
                Sell[i] = signal['Close'][i]
                flag = 0

    return (Short, Sell)

In [463]:
THRESHOLD = 0.7

RSI_news_input = prepare_dataset(path, ['Positive', 'Negative', 'Neutral'])
RSI_news_input = pd.merge(RSI_news_input, stocks[['Date', 'RSI']], on='Date', how='left')

columns = ['Positive', 'Negative', 'Neutral']

for col in columns:
    RSI_news_input[col] = RSI_news_input[col].apply(lambda arr: replace_below_threshold(arr, threshold=THRESHOLD))        
    
RSI_news_input[['Positive', 'Negative', 'Neutral']] = RSI_news_input[['Positive', 'Negative', 'Neutral']].applymap(len)

RSI_news_input['Predictions'] = RSI_news_input.apply(take_max_count_prediction, axis=1)

print("Threshold : ", THRESHOLD, " Long Cumulative Return : ", cumulative_reward_long(RSI_news_input, buy_sell_RSI_sentiment))
print("Threshold : ", THRESHOLD, " Short Cumulative Return : ", cumulative_reward_short(RSI_news_input, short_sell_RSI_sentiment))

Threshold :  0.7  Long Cumulative Return :  (28.151815328466427, 5.28135712185228)
Threshold :  0.7  Short Cumulative Return :  (-2.267414514192126, 5.498683313301388)
